## The Template pattern

(sometimes called the Template method) is useful
for removing duplicate code; it's intended to support the Don't Repeat
Yourself principle. It is designed for situations where we have several different tasks to
accomplish that have some, but not all, steps in common. The common steps are
implemented in a base class, and the distinct steps are overridden in subclasses to
provide custom behavior.

![](uml/template_pattern.png)

### A Template example
We can use the Template pattern to put the common
steps in a base class, and the varying steps in two subclasses.

In [1]:
import contextlib
import csv
from pathlib import Path
import sqlite3
from typing import ContextManager, TextIO, cast, Optional
import sys


def test_setup(db_name: str = "sales.db") -> None:
    conn = sqlite3.connect(db_name)

    conn.execute(
        """
        CREATE TABLE IF NOT EXISTS Sales (
            salesperson text,
            amt currency,
            year integer,
            model text,
            new boolean
        )
        """
    )

    conn.execute(
        """
        DELETE FROM Sales
        """
    )

    conn.execute(
        """
        INSERT INTO Sales 
        VALUES('Tim', 16000, 2010, 'Honda Fit', 'true')
        """
    )
    conn.execute(
        """
        INSERT INTO Sales 
        VALUES('Tim', 9000, 2006, 'Ford Focus', 'false')
        """
    )
    conn.execute(
        """
        INSERT INTO Sales 
        VALUES('Hannah', 8000, 2004, 'Dodge Neon', 'false')
        """
    )
    conn.execute(
        """
        INSERT INTO Sales 
        VALUES('Hannah', 28000, 2009, 'Ford Mustang', 'true')
        """
    )
    conn.execute(
        """
        INSERT INTO Sales 
        VALUES('Hannah', 50000, 2010, 'Lincoln Navigator', 'true')
        """
    )
    conn.execute(
        """
        INSERT INTO Sales 
        VALUES('Jason', 20000, 2008, 'Toyota Prius', 'false')
        """
    )
    conn.commit()
    conn.close()
    
test_setup()

In [2]:
import abc

class QueryTemplate(abc.ABC):
    def __init__(self, db_name: str = "sales.db") -> None:
        self.db_name = db_name
        self.conn: sqlite3.Connection
        self.results: list[tuple[str, ...]]
        self.query: str
        self.header: list[str]
        
    def connect(self) -> None:
        self.conn = sqlite3.connect(self.db_name)
    
    @abc.abstractmethod
    def construct_query(self) -> None: ...
        
    def do_query(self) -> None:
        results = self.conn.execute(self.query)
        self.results = results.fetchall()
        self.conn.close()
    
    def output_context(self) -> ContextManager[TextIO]:
        self.target_file = sys.stdout
        return cast(ContextManager[TextIO], contextlib.nullcontext())
            
    def output_results(self) -> None:
        writer = csv.writer(self.target_file)
        writer.writerow(self.header)
        writer.writerows(self.results)
        
    def process_format(self) -> None:
        self.connect()
        self.construct_query()
        self.do_query()
        with self.output_context():
            self.output_results()


In [3]:
class NewVehicleQuery(QueryTemplate):
    def construct_query(self) -> None: 
        self.query = """
            SELECT * FROM Sales WHERE new='true'
        """
        self.header = ["salesperson", "amt", "year", "model", "new"]
        
task_1 = NewVehicleQuery()
task_1.process_format()

salesperson,amt,year,model,new
Tim,16000,2010,Honda Fit,true
Hannah,28000,2009,Ford Mustang,true
Hannah,50000,2010,Lincoln Navigator,true


In [4]:
class SelesGrossQuery(QueryTemplate):
    def construct_query(self) -> None:
        self.query = """
            SELECT salesperson, sum(amt) FROM Sales GROUP BY salesperson
        """
        self.header = ["salesperson", "total sales"]
        
    def output_context(self) -> ContextManager[TextIO]:
        filepath = Path(f"gross_sales.csv")
        self.target_file = filepath.open("w")
        return self.target_file
    
task2 = SelesGrossQuery()
task2.process_format()

with open("gross_sales.csv") as file:
    print(file.read())

salesperson,total sales
Hannah,86000
Jason,20000
Tim,25000



In [5]:
if __name__ == '__main__':        
    import doctest
    import subprocess
    name = "16-The Template pattern"
    doctest.testmod(verbose=False)
    subprocess.run(f'jupyter nbconvert --to script --output test "{name}"', shell=True)
    std_out = subprocess.run('mypy --strict test.py', capture_output=True, shell=True).stdout
    print(std_out.decode('ascii'))

[NbConvertApp] Converting notebook 16-The Template pattern.ipynb to script
[NbConvertApp] Writing 4469 bytes to test.py


Success: no issues found in 1 source file

